# Vehicle Tracking & Line Counting (YOLOv11 + BoT-SORT)

This notebook mirrors the “real-time vehicle tracking and counting” tutorial flow, but uses **YOLOv11** with **BoT-SORT** and writes a single annotated video with **line-based counts** (no phases).

**What it does:**
- Detect vehicles (car, motorcycle, bus, truck)
- Track them with BoT-SORT (IDs)
- Count crossings over **three horizontal line segments** A/B/C
- Save the result to `vehicle_lines_counted.mp4`



## 0) Install packages

In [ ]:
!pip -q install ultralytics opencv-python shapely lapx

## 1) Configuration

In [10]:
from collections import defaultdict, deque
from pathlib import Path
import numpy as np
import math, cv2, time
from ultralytics import YOLO

# --- Files ---
WEIGHTS = 'yolo11m.pt'    # try yolo11s/m for stability, or yolo11n for speed
SOURCE  = 'vehicles.mp4'         # same naming as many tutorials; set to your video path or 0 for webcam

# --- Classes (COCO) ---
# car=2, motorcycle=3, bus=5, truck=7
VEHICLE_CLASSES = [2, 3, 5, 7]

# --- Detection params ---
CONF  = 0.35
IOU   = 0.5
IMGSZ = 960

# --- Line segments (A/B/C) ---
# Adjust these to match your video frame geometry.
# Format: (x1, y1), (x2, y2); here: three horizontal segments near y=480
start_line_A = (0,   480); end_line_A = (480,  480)
start_line_B = (525, 480); end_line_B = (745,  480)
start_line_C = (895, 480); end_line_C = (1165, 480)

# Colors (BGR)
COLOR_A = (0, 255, 0)     # green
COLOR_B = (255, 0, 0)     # blue 
COLOR_C = (0, 0, 255)     # red
COLOR_ID = (0, 170, 255)  # amber for boxes/IDs
COLOR_TXT = (255, 255, 255)

# Counters
counter_A = 0
counter_B = 0
counter_C = 0

# Trajectories: track_id -> deque of recent center points
points = defaultdict(lambda: deque(maxlen=32))

print('Config OK:', WEIGHTS, SOURCE)

Config OK: yolo11m.pt vehicles.mp4


## 2) Run — Detect + Track + Line Counting → `vehicle_lines_counted.mp4`

In [11]:
# Load model
model = YOLO(WEIGHTS)

# Start tracking stream (use BoT-SORT by default; you can switch to 'deepsort.yaml')
results = model.track(
    source=SOURCE,
    tracker='botsort.yaml',         # or 'deepsort.yaml'
    stream=True,
    classes=VEHICLE_CLASSES,        # keep vehicle classes only
    conf=CONF,
    iou=IOU,
    imgsz=IMGSZ,
    persist=True,
    verbose=False,
)

writer = None
frames = 0

global counter_A, counter_B, counter_C

# Map COCO class IDs to nice names
NAME_MAP = {2: "Car", 3: "Motorcycle", 5: "Bus", 7: "Truck"}

for r in results:
    # Base frame
    frame = r.orig_img.copy()
    overlay = frame.copy()

    # Draw line segments A/B/C (thick for visibility)
    cv2.line(frame, start_line_A, end_line_A, COLOR_A, 12)
    cv2.line(frame, start_line_B, end_line_B, COLOR_B, 12)
    cv2.line(frame, start_line_C, end_line_C, COLOR_C, 12)

    # Semi-transparent blend (like tutorial)
    frame = cv2.addWeighted(overlay, 0.5, frame, 0.5, 0)

    # Lazy-init writer on first frame
    if writer is None:
        h, w = frame.shape[:2]
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        writer = cv2.VideoWriter('vehicle_lines_counted.mp4', fourcc, 30.0, (w, h))

    # Collect tracked vehicle boxes + IDs
    current_ids = []
    if r.boxes is not None and len(r.boxes) > 0 and getattr(r.boxes, 'id', None) is not None:
        ids = r.boxes.id.int().cpu().numpy()
        cls = r.boxes.cls.int().cpu().numpy()
        # mask to allowed vehicles
        keep = np.isin(cls, np.array(VEHICLE_CLASSES))
        xyxy = r.boxes.xyxy.cpu().numpy()[keep]
        ids_f = ids[keep]
        cls_f = cls[keep]  # <-- aligned class IDs for kept detections

        # draw and update points
        for (x1, y1, x2, y2), tid, cid in zip(xyxy, ids_f, cls_f):
            tid = int(tid)
            class_name = NAME_MAP.get(int(cid), str(int(cid)))  # fallback to numeric if unmapped

            # draw box + "<id> <Class>" label (e.g., "1 Car")
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), COLOR_ID, 3)
            pos = (int(x1), max(0, int(y1) - 5))  # ensure a proper (x, y) tuple of ints
            cv2.putText(frame, f"{tid} {class_name}",
                        pos,
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, COLOR_ID, 2)

            # center point
            cx, cy = int((x1 + x2) / 2), int((y1) + (y2 - y1) / 2)
            points[tid].append((cx, cy))
            cv2.circle(frame, (cx, cy), 4, (0, 255, 0), -1)

            # draw small trajectory
            pt_list = points[tid]
            for i in range(1, len(pt_list)):
                p1 = pt_list[i - 1]; p2 = pt_list[i]
                if p1 is None or p2 is None: continue
                cv2.line(frame, p1, p2, (0, 255, 0), 2)

            # crossing logic 
            # get earliest stored point (oldest) as "last_point" along the path
            if len(points[tid]) >= 2:
                last_point_x = points[tid][0][0]
                last_point_y = points[tid][0][1]

                # A segment
                if cy > start_line_A[1] and start_line_A[0] < cx < end_line_A[0] and last_point_y < start_line_A[1]:
                    counter_A += 1
                    points[tid].clear()  # reset path to avoid double-counts

                # B segment
                elif cy > start_line_B[1] and start_line_B[0] < cx < end_line_B[0] and last_point_y < start_line_B[1]:
                    counter_B += 1
                    points[tid].clear()

                # C segment
                elif cy > start_line_C[1] and start_line_C[0] < cx < end_line_C[0] and last_point_y < start_line_C[1]:
                    counter_C += 1
                    points[tid].clear()

    # HUD: show current counts and labels for A/B/C
    cv2.putText(frame, "A", (10, 483), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLOR_TXT, 2)
    cv2.putText(frame, "B", (530, 483), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLOR_TXT, 2)
    cv2.putText(frame, "C", (910, 483), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLOR_TXT, 2)
    cv2.putText(frame, f"{counter_A}", (270, 483), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLOR_TXT, 2)
    cv2.putText(frame, f"{counter_B}", (620, 483), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLOR_TXT, 2)
    cv2.putText(frame, f"{counter_C}", (1040, 483), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLOR_TXT, 2)

    # Write frame
    writer.write(frame)
    frames += 1

# Cleanup
if writer is not None:
    writer.release()

print(f"Saved: vehicle_lines_counted.mp4 ({frames} frames)")


Saved: vehicle_lines_counted.mp4 (2112 frames)
